<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [1]:
#Start Here
import re
import numpy as np
import pandas as pd
from pprint import pprint
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

# Gensim
import gensim
from gensim import models
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
# don't skip this
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# Download spacy model
import spacy.cli
spacy.cli.download("en_core_web_lg")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [2]:
# You need to unzip the file from module 1 first and bring it to the data folder in module 4
df = pd.read_csv('./data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [3]:
df['reviews.text']

0        I order 3 of them and one of the item is bad q...
1        Bulk is always the less expensive way to go fo...
2        Well they are not Duracell but for the price i...
3        Seem to work as well as name brand batteries a...
4        These batteries are very long lasting the pric...
                               ...                        
28327    I got 2 of these for my 8 yr old twins. My 11 ...
28328    I bought this for my niece for a Christmas gif...
28329    Very nice for light internet browsing, keeping...
28330    This Tablet does absolutely everything I want!...
28331    At ninety dollars, the expectionations are low...
Name: reviews.text, Length: 28332, dtype: object

In [4]:
df['reviews.text'] = df['reviews.text'].apply(lambda x: re.sub('[^0-9 a-zA-Z]', '', x))
df['reviews.text']

0        I order 3 of them and one of the item is bad q...
1        Bulk is always the less expensive way to go fo...
2        Well they are not Duracell but for the price i...
3        Seem to work as well as name brand batteries a...
4        These batteries are very long lasting the pric...
                               ...                        
28327    I got 2 of these for my 8 yr old twins My 11 y...
28328    I bought this for my niece for a Christmas gif...
28329    Very nice for light internet browsing keeping ...
28330    This Tablet does absolutely everything I want ...
28331    At ninety dollars the expectionations are low ...
Name: reviews.text, Length: 28332, dtype: object

In [5]:
nlp = spacy.load('en_core_web_lg')
def lemma_producer(x):
    """
    tokenizes string, returning a list of lemmas
    """
    lemmas = []
    for token in nlp(x):
        if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_!= 'PRON'):
            lemmas.append(token.lemma_)
    return lemmas
df['lemmas'] = df['reviews.text'].apply(lemma_producer)
df['lemmas'][0]

['order',
 '3',
 'item',
 'bad',
 'quality',
 'miss',
 'backup',
 'spring',
 'pc',
 'aluminum',
 'battery',
 'work']

In [6]:
df['lemmas'].iloc[100]

['teacher',
 'need',
 'ton',
 'battery',
 'refuse',
 'spend',
 'excessive',
 'amount',
 'figure',
 'good',
 'option',
 'longlaste',
 'worth',
 'money',
 'cute',
 'matter',
 'lol',
 'will',
 'not',
 'find',
 'deal',
 'like',
 'storesI',
 'highly',
 'recommend']

In [7]:
# Create Dictionary
id2word = corpora.Dictionary(df['lemmas'])

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in df['lemmas']]

id2word[300]

'997'

In [8]:
df['reviews.text'][5]

'Bought a lot of batteries for Christmas and the AmazonBasics Cell have been good I havent noticed a difference between the brand name batteries and the Amazon Basic brand Just a lot easier to purchase and have arrive at the house and have on hand Will buy again'

In [9]:
id2word[255]

'everyday'

In [10]:
corpus[5]

[(4, 2),
 (20, 2),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 2),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 2),
 (38, 1),
 (39, 1),
 (40, 1)]

In [11]:
# Human readable format of corpus (term-frequency)
[(id2word[word_id], word_count) for word_id, word_count in corpus[5]]

[('battery', 2),
 ('brand', 2),
 ('Amazon', 1),
 ('AmazonBasics', 1),
 ('Basic', 1),
 ('Cell', 1),
 ('Christmas', 1),
 ('arrive', 1),
 ('buy', 2),
 ('difference', 1),
 ('easy', 1),
 ('good', 1),
 ('hand', 1),
 ('house', 1),
 ('lot', 2),
 ('not', 1),
 ('notice', 1),
 ('purchase', 1)]

In [12]:
lda_multicore = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                                        id2word=id2word,
                                                        num_topics=20, 
                                                        chunksize=100,
                                                        passes=10,
                                                        per_word_topics=True,
                                                        workers=12)

In [13]:
lda_multicore.save('lda_multicore.model')
lda_multicore =  models.LdaModel.load('lda_multicore.model')
pprint(lda_multicore.print_topics())
doc_lda = lda_multicore[corpus]

[(0,
  '0.055*"not" + 0.043*"free" + 0.031*"light" + 0.030*"Amazon" + '
  '0.024*"memory" + 0.022*"card" + 0.020*"add" + 0.016*"read" + 0.015*"like" + '
  '0.015*"purchase"'),
 (1,
  '0.062*"purchase" + 0.058*"Amazon" + 0.052*"fast" + 0.047*"happy" + '
  '0.040*"product" + 0.030*"buy" + 0.030*"great" + 0.029*"picture" + '
  '0.029*"need" + 0.028*"item"'),
 (2,
  '0.346*"good" + 0.133*"screen" + 0.114*"price" + 0.059*"quality" + '
  '0.034*"product" + 0.023*"clear" + 0.016*"buy" + 0.016*"high" + 0.016*"low" '
  '+ 0.013*"battery"'),
 (3,
  '0.200*"perfect" + 0.169*"size" + 0.065*"right" + 0.060*"break" + '
  '0.034*"hand" + 0.032*"pleased" + 0.032*"super" + 0.028*"fit" + '
  '0.026*"perfectly" + 0.022*"receive"'),
 (4,
  '0.298*"app" + 0.035*"store" + 0.035*"good" + 0.032*"pretty" + '
  '0.027*"laptop" + 0.025*"wish" + 0.025*"Amazon" + 0.021*"wonderful" + '
  '0.020*"stuff" + 0.019*"need"'),
 (5,
  '0.308*"love" + 0.078*"buy" + 0.076*"Christmas" + 0.067*"nice" + 0.031*"new" '
  '+ 0.027

In [14]:
# Compute Perplexity
print('\nPerplexity: ', lda_multicore.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_multicore, 
                                     texts=df['lemmas'], 
                                     dictionary=id2word, 
                                     coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -10.987924451470601

Coherence Score:  0.4207668806278163


In [ ]:
# Could've probably had a better score if I had adjusted the number of topics to it's ideal parameter.

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling